# Parker Dunn

__Assignment for COURSERA: Introduction to Deep Learning (via CU Boulder)__  
__Assignment:__ Week 3 - CNN Cancer Detection Kaggle Mini-Project


## To be added to main file

Forgot to push the most recent version of my script from home.

## Section 3 - Model Architecture

#### Plan

Due to limited time and computing resources, I'll stick to a simple model. I plan to use the "building block-style" Covolution-Convolution-Pooling design pattern with probably no more than 4 repetitions of this pattern. Since we previously experimented with the development of neural network architecture, I am hoping to replicate a reliable NN structure from one of the example image classification models from the videos. In theory, the key features will be extracted by the convolution architecture and the NN structure from another image classification task can be successfully optimized for new features.

Laid out below are my achitecture plans as well as some of the thoughts I have regarding the training of my CNN.

__Design parameters and Hyperparameters__
Decisions
* I will use ReLU (hidden layers) and sigmoid (output layer) as activation functions. This is not a design parameter that I plan to vary this time.
* I will primarily use 3x3xd convolutional filters
* As an optimization method, I will stick to SGD, which I am most familiar with, and plant to incorporate momentum if possible with the Keras API.

Hyperparameters
* Learning rate
    * Test: 0.01 | 0.001 | 0.0001 (3 values)
* Momentum
    * Test: 0.0 | 0.01 | 0.1 (3 values)
* Number of epochs (i.e., how much training)

Design
* Number of [Conv-Conv-Pool] layers
    - Test: 2, 3, 4
* Number of filters to use
* 

Potential ways to improve a struggling model
* L2 regularization
* Batch normalization

I plan to use moderate training parameters at first (e.g. learning rate -> 0.001 and momentum -> 0.01) to experiment and narrow down some viable convolution designs.

#### Step 3 - Part 1: Trying to find a repeatable way to create a CNN!

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

__Inputs__
* List of layers
    - convolution layers -> need filter sizes too
    - NN layers -> any regularization?

__Outputs__
* Model object

In [ ]:
conv_design = ["conv", (3,3,3), "conv", (3,3,3), "maxpool"]
nn_architecture = [92, 48, 12, 1] # completely guessed that these are reasonable numbers

model = tf.keras.Sequential()

